In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Load IMDB Dataset
vocab_size = 10000  # Use top 10,000 words
maxlen = 200        # Maximum review length

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
# Pad Sequences
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
# Function to create models
def build_binary_model(recurrent_layer, name="Model"):
    model = Sequential(name=name)
    model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=maxlen))
    model.add(recurrent_layer)
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# RNN Model
rnn_binary_model = build_binary_model(SimpleRNN(64), name="rnn_binary_model")
rnn_binary_model.summary()
rnn_binary_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)
rnn_acc = rnn_binary_model.evaluate(x_test, y_test)[1]
# LSTM Model
lstm_binary_model = build_binary_model(LSTM(64), name="lstm_binary_model")
lstm_binary_model.summary()
lstm_binary_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)
lstm_acc = lstm_binary_model.evaluate(x_test, y_test)[1]
# GRU Model
gru_binary_model = build_binary_model(GRU(64), name="gru_binary_model")
gru_binary_model.summary()
gru_binary_model.fit(x_train, y_train, epochs=3, batch_size=64, validation_split=0.2)
gru_acc = gru_binary_model.evaluate(x_test, y_test)[1]
# Print comparison
print(f"Accuracy Comparison:")
print(f"Simple RNN Accuracy: {rnn_acc:.4f}")
print(f"LSTM Accuracy     : {lstm_acc:.4f}")
print(f"GRU Accuracy      : {gru_acc:.4f}")
# Load IMDB word index to decode reviews
word_index = imdb.get_word_index()
reverse_word_index = {value + 3: key for key, value in word_index.items()}
reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNK>"
reverse_word_index[3] = "<UNUSED>"

# Decode a review from integer indices
def decode_review(encoded_review):
    return " ".join([
        reverse_word_index.get(i, "?")
        for i in encoded_review
        if i >= 4  # skip <PAD>, <START>, <UNK>, <UNUSED>
    ])

# Select test samples
num_samples = 10
test_reviews = x_test[:num_samples]
true_labels = y_test[:num_samples]

# Predict with each model
rnn_preds = (rnn_binary_model.predict(test_reviews) > 0.5).astype("int32").flatten()
lstm_preds = (lstm_binary_model.predict(test_reviews) > 0.5).astype("int32").flatten()
gru_preds = (gru_binary_model.predict(test_reviews) > 0.5).astype("int32").flatten()

# Construct the comparison table
results = []
for i in range(num_samples):
    results.append({
        "Review (truncated)": decode_review(test_reviews[i])[:300] + "...",
        "Actual": "Positive" if true_labels[i] == 1 else "Negative",
        "SimpleRNN Prediction": ("Pos" if rnn_preds[i] == 1 else "Neg") + (" (Hit)" if rnn_preds[i] == true_labels[i] else " (Miss)"),
        "LSTM Prediction": ("Pos" if lstm_preds[i] == 1 else "Neg") + (" (Hit)" if lstm_preds[i] == true_labels[i] else " (Miss)"),
        "GRU Prediction": ("Pos" if gru_preds[i] == 1 else "Neg") + (" (Hit)" if gru_preds[i] == true_labels[i] else " (Miss)")
    })

# Convert to DataFrame and show
comparison_df = pd.DataFrame(results)
print(comparison_df)